In [1]:
import tensorflow

In [3]:
import os

In [4]:
%pwd

'e:\\Kidney-Disease-Cassification\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'e:\\Kidney-Disease-Cassification'

In [18]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list 
    params_include_top: bool
    params_weights: str
    params_classes: int 
    

In [9]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,create_directories 

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_fiepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_fiepath) 
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig: 
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])    
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path), 
            params_image_size=self.params.IMAGE_SIZE, 
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        
        return prepare_base_model_config     

In [14]:
import os
import urllib.request as request   
from zipfile import ZipFile
import tensorflow as tf  

In [37]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )   
        
        self.save_model(path=self.config.base_model_path,model=self.model) 
        
    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)  
        
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False   
                
        flatten_in = tf.keras.layers.Flatten()(model.output) 
        norm = tf.keras.layers.BatchNormalization()(flatten_in) 
        temp = tf.keras.layers.Dense(
            units = 32,
            activation="relu"
        )(flatten_in) 
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation="softmax"
        )(temp)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )  
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"] 
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None   
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
        
    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)          

In [38]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e    

[2023-12-15 21:27:29,038: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 21:27:29,043: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 21:27:29,046: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-12-15 21:27:30,302: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                     

In [ ]:
Path